# Himmelblau Benchmark
## Imports

In [ ]:
import os

import numpy as np
import pandas as pd
import plotly.express as px

import bofire.strategies.api as strategies
from bofire.benchmarks.single import Himmelblau
from bofire.data_models.acquisition_functions.api import qLogEI
from bofire.data_models.api import Domain
from bofire.data_models.strategies.api import RandomStrategy, SoboStrategy, TuRBOConfig
from bofire.runners.api import run


SMOKE_TEST = os.environ.get("SMOKE_TEST")

## Random Optimization

In [ ]:
def sample(domain):
    datamodel = RandomStrategy(domain=domain)
    sampler = strategies.map(data_model=datamodel)
    sampled = sampler.ask(10)
    return sampled


def best(domain: Domain, experiments: pd.DataFrame) -> float:
    return experiments.y.min()


random_results = run(
    Himmelblau(),
    strategy_factory=lambda domain: strategies.map(RandomStrategy(domain=domain)),
    n_iterations=50 if not SMOKE_TEST else 1,
    metric=best,
    initial_sampler=sample,
    n_runs=1,
    n_procs=1,
)

Run 0: 100%|██████████| 50/50 [00:00<00:00, 238.69it/s, Current Best:=7.070]


## SOBO (GPEI) Optimization

In [ ]:
def strategy_factory(domain: Domain):
    data_model = SoboStrategy(domain=domain, acquisition_function=qLogEI())
    return strategies.map(data_model)


bo_results = run(
    Himmelblau(),
    strategy_factory=strategy_factory,
    n_iterations=50 if not SMOKE_TEST else 1,
    metric=best,
    initial_sampler=sample,
    n_runs=1,
    n_procs=1,
)

Run 0: 100%|██████████| 50/50 [00:10<00:00,  4.63it/s, Current Best:=0.032]


## SOBO (GPEI+TR) Optimization

In [ ]:
def strategy_factory(domain: Domain):
    data_model = SoboStrategy(
        domain=domain, acquisition_function=qLogEI(), trust_region_config=TuRBOConfig()
    )
    return strategies.map(data_model)


tr_results = run(
    Himmelblau(),
    strategy_factory=strategy_factory,
    n_iterations=50 if not SMOKE_TEST else 1,
    metric=best,
    initial_sampler=sample,
    n_runs=1,
    n_procs=1,
)

Run 0: 100%|██████████| 50/50 [06:05<00:00,  7.31s/it, Current Best:=0.001]


## Plot the results

In [ ]:
df_results = pd.concat(
    [random_results[0][0]["y"], bo_results[0][0]["y"], tr_results[0][0]["y"]], axis=1
)
df_results.columns = ["Random", "SOBO", "SOBO+TR"]
fig_min = px.line(
    np.minimum.accumulate(df_results).loc[10:], title="Himmelblau Benchmark"
)
fig_min.show()